In [1]:
import os
os.environ['USE_PYGEOS'] = '0' # 切换到shapely 2.0计算后端

import shapely
import numpy as np
import geopandas as gpd
import dask_geopandas as dgpd
from shapely import Point, Polygon

print(
    'geopandas: ', gpd.__version__,
    'dask_geopandas: ', dgpd.__version__,
    'shapely: ', shapely.__version__
)

geopandas:  0.12.2 dask_geopandas:  v0.3.0 shapely:  2.0.0


# 2 dask-geopandas的使用

## 2.1 基础使用

In [2]:
# 生成示例用矢量数据
demo_points = gpd.GeoDataFrame(
    {
        'id': range(1000000),
        'geometry': [
            Point(np.random.uniform(0, 90),
                  np.random.uniform(0, 90))
            for i in range(1000000)
        ]
    }
)

# 写出到本地gdb
demo_points.to_file('./demo_points.gdb', driver='OpenFileGDB')

In [3]:
demo_points = gpd.read_file('./demo_points.gdb', driver='OpenFileGDB')
demo_points_ddf = dgpd.from_geopandas(demo_points, npartitions=4)
demo_points_ddf

,id,geometry
npartitions=4,,
0,float64,geometry
250000,...,...
500000,...,...
750000,...,...
999999,...,...


In [4]:
demo_points_ddf.id.max()

dd.Scalar<series-..., dtype=float64>

In [5]:
demo_points_ddf.id.max().compute()

999999.0

In [6]:
demo_points_ddf.geometry.distance(Point(0, 0))

Dask Series Structure:
npartitions=4
0         float64
250000        ...
500000        ...
750000        ...
999999        ...
dtype: float64
Dask Name: distance, 3 graph layers

In [7]:
demo_points_ddf.geometry.distance(Point(0, 0)).compute()

0          75.881400
1          26.118355
2          51.879331
3          59.875140
4          88.866753
             ...    
999995     82.103003
999996     67.935746
999997    107.640030
999998     87.186818
999999     26.534879
Length: 1000000, dtype: float64

## 2.2 性能比较

In [8]:
%%timeit 

demo_points.geometry.distance(Point(0, 0))

351 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit 

demo_points_ddf.geometry.distance(Point(0, 0)).compute()

113 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit 

demo_points.within(Polygon([(0, 0), (0, 10), (10, 10), (10, 0)]))

52.9 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit

demo_points_ddf.within(Polygon([(0, 0), (0, 10), (10, 10), (10, 0)])).compute()

19.6 ms ± 799 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
